In [1161]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

<div class="alert" style="background:#195734;color:#FFFFFF">
  <center><h1> <strong> Welcome to Clinical Decision Support System Powered by OpenMRS </strong> </h1> </center>
</div>

In [1162]:
# from IPython.display import IFrame
# IFrame("http://149.165.238.106:8081/notebooks/openmrs_cdss_dashboard.ipynb", 900,500)

In [1163]:
# %javascript
# // // Send this back as a python variable (this just executes python statements)
# // IPython.notebook.kernel.execute('patientId="' + new URL(window.location.href).searchParams.get("patientId") + '";');

In [1164]:
# import sys
# !{sys.executable} -m pip install --user seaborn
# !{sys.executable} -m pip install --user ipywidgets
# !{sys.executable} -m pip install --user pandas
# !{sys.executable} -m pip install --user matplotlib
# !{sys.executable} -m pip install --user scipy
# !{sys.executable} -m pip install --user sklearn
# !{sys.executable} -m pip install --user fastkml
# !{sys.executable} -m pip install --user folium
# !{sys.executable} -m pip install --user tabulate

from IPython.display import display, clear_output
from IPython.display import Markdown

import IPython.display
import pandas as pd
import numpy as np
import fastai
from fastai.tabular.all import *

import ipywidgets as widgets
from ipywidgets import Button, Layout
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Box


import warnings
warnings.filterwarnings('ignore', category = FutureWarning)


from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
from pandas.plotting import table

from scipy import stats
from pingouin import ttest
import seaborn as sns
import zipfile
from fastkml.kml import KML
import folium
import datetime
from datetime import date
import sklearn as sklearn
import scikitplot as skplt

from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler



from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb 
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import MySQLdb
from pandas.io import sql 

from simple_colors import *

In [1165]:
# Setting up the database connection

conn = MySQLdb.connect(host="localhost", user="root", passwd="Regina", db="openmrs") 
openmrscursor = conn.cursor()  

In [1166]:
# selecting the diagnosis

In [1167]:
## Patient Attributes
## 12576264, 10152346
## Patient Attributes
patient_id = 10152346
values_mapx = {}
values_mapx['patient_id'] = patient_id 


In [1168]:
# creating cohort
def on_1button_clicked(selected_diag):
    print(selected_diag)
    diag_con = selected_diag
    diag_con = list(filter(None,diag_con))
    values_mapx['diag_con'] = diag_con
    patient_cohort   = (""" select p.person_id from person p, mimic_openmrs_cdss cdss where p.person_id
                            IN (select patient_id from encounter_diagnosis where diagnosis_coded 
                            IN (select concept_id from concept where description 
                            IN {diag_con}))
                            AND cdss.person_id=p.person_id 
                            AND p.gender= (select p.gender from person where person_id = {patient_id}) 
                            AND age_group_code=(select age_group_code from mimic_openmrs_cdss where person_id = {patient_id})""")
    patient_cohort = patient_cohort.format(**values_mapx).replace('[', '(').replace(']',')')
    patient_cohort = pd.read_sql(patient_cohort, con=conn)
    print(str(patient_cohort.shape[0]) + " Similar Patients For This Diagnosis")
    #display("Found " + str(patient_cohort.shape[0]) + " similar patients")
    #dropdown_diagconcept_eventhandler(on_1button_clicked)


 <div class="card" style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);max-width: 100%; margin: auto;padding:10px; font-family: arial; background:#D9D9D9">
    <p>Please choose <b style="color:#195734">ONE DIAGNOSIS</b> to run the analysis:</p>
 </div>

In [1169]:
# from IPython.display import display,Markdown

# display(Markdown('<div class="card" style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);max-width: 100%; margin: auto;padding:10px; font-family: arial;"><p>Please choose <b style="color:#009384">ONE DIAGNOSIS</b> to run the analysis:</p>'print(5)'</div>'))


In [1170]:
# selecting the diagnosis
style = {'description_width': 'initial'}
diagnames_pat_id = "SELECT description from concept where concept_id IN (SELECT * FROM(SELECT diagnosis_coded FROM encounter_diagnosis WHERE patient_id = {patient_id}) as xyz)" 
diagnames_pat_id = diagnames_pat_id.format(**values_mapx)
diagnames_pat_id = pd.read_sql(diagnames_pat_id, con=conn)
diagnames_value = diagnames_pat_id['description']

output_diagnames_pat_id = widgets.Output()
dropdown_diagnames_pat_id = widgets.SelectMultiple(options= diagnames_pat_id['description'].unique(), style= style,disabled=False, rows = 2, size = 10, layout=Layout(width = '440px'))

def dropdown_diagconcept_eventhandler(change):
    output_diagnames_pat_id.clear_output()
    with output_diagnames_pat_id:
        display(diagnames_pat_id[diagnames_pat_id['description'] == change.new])
    on_1button_clicked(dropdown_diagnames_pat_id.value)
     
dropdown_diagnames_pat_id.observe(dropdown_diagconcept_eventhandler, names='value')
display(dropdown_diagnames_pat_id)

/tmp/ipykernel_45791/3097201369.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  diagnames_pat_id = pd.read_sql(diagnames_pat_id, con=conn)


SelectMultiple(layout=Layout(width='440px'), options=('Pneumonia, unspecified organism', 'Viral intestinal inf…

('Pneumonia, unspecified organism',)


/tmp/ipykernel_45791/4176199392.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patient_cohort = pd.read_sql(patient_cohort, con=conn)


DatabaseError: Execution failed on sql:  select p.person_id from person p, mimic_openmrs_cdss cdss where p.person_id
                            IN (select patient_id from encounter_diagnosis where diagnosis_coded 
                            IN (select concept_id from concept where description 
                            IN ('Pneumonia, unspecified organism')))
                            AND cdss.person_id=p.person_id 
                            AND p.gender= (select p.gender from person where person_id = 10152346) 
                            AND age_group_code=(select age_group_code from mimic_openmrs_cdss where person_id = 10152346)
(4031, 'The client was disconnected by the server because of inactivity. See wait_timeout and interactive_timeout for configuring this behavior.')
unable to rollback

In [1171]:
patient_attribute_df = pd.read_sql("""select p.gender, m.age_group_code, c.concept_id, o.concept_id, cn.name,  c.description, 
                                e.diagnosis_coded from person as p LEFT JOIN mimic_openmrs_cdss as m ON p.person_id = m.person_id 
                                LEFT JOIN obs as o ON o.person_id = p.person_id  LEFT JOIN concept_name as cn ON cn.concept_id = o.concept_id 
                                LEFT JOIN encounter_diagnosis as e ON e.patient_id = p.person_id LEFT JOIN concept as c ON 
                                c.concept_id = e.diagnosis_coded WHERE  p.person_id = {}""".format(patient_id), con = conn)

/tmp/ipykernel_45791/3557023205.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patient_attribute_df = pd.read_sql("""select p.gender, m.age_group_code, c.concept_id, o.concept_id, cn.name,  c.description,


In [1172]:
def on_3button_clicked(b):
    output_count = widgets.Output()
    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)

# concept_name = ['creatinine', 'albumin']
    values_map = {}

    a = 0
    final_patient_ind_value_numeric = pd.DataFrame(columns = ['person_id'])
    final_patient_ind_value_numeric['person_id'] = final_patient_ind_value_numeric['person_id'].astype(int)

    dfss = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1

        patient_value_numeric = (""" select o.person_id, o.value_numeric from obs as o where o.concept_id IN (SELECT c.concept_id FROM concept as c WHERE c.description = '{concept_name}') AND o.person_id = {patient_id}""")
        patient_value_numeric = patient_value_numeric.format(**values_map)

        patient_ind_value_numeric = pd.read_sql(patient_value_numeric, con = conn)
    
        patient_ind_value_numeric.columns = [*patient_ind_value_numeric.columns[:-1], i]
    
        name = "patient_ind_value_numeric" + i
     
        locals()[name] = patient_ind_value_numeric.copy()

        dfss.append(locals()[name])

    ls = [df.set_index('person_id') for df in dfss]

# pd.DataFrame.join accepts a list of dataframes as 'other'
    final_patient_ind_value_numeric = ls[0].append(ls[1:])
# final_patient_ind_value_numeric = ls[0].join(ls[1:], how = 'inner')
    final_patient_ind_value_numeric = final_patient_ind_value_numeric.reset_index(drop = False)

    for column in final_patient_ind_value_numeric.columns:
        final_patient_ind_value_numeric[column] = final_patient_ind_value_numeric[column].fillna(final_patient_ind_value_numeric[column].mean())

    def individual_patient_value():
        display(final_patient_ind_value_numeric)
    individual_patient_value() 

In [1173]:
def patient_cohort_concept_():
    output_count = widgets.Output()

    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)
    concept_name

# concept_name = ['creatinine', 'albumin']
    values_map = {}
    a = 0
# final_patient_cohort_concept = pd.DataFrame(columns = ['person_id'])
# final_patient_cohort_concept['person_id'] = final_patient_cohort_concept['person_id'].astype(int)

    dfs = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1
    
        sql_query = ("""select person_id, value_numeric from obs where person_id IN (select p.person_id from person p, 
                mimic_openmrs_cdss cdss where p.person_id IN (select patient_id from encounter_diagnosis where 
                diagnosis_coded IN (select c.concept_id from concept c where c.description LIKE 
                ('Pneumonia, unspecified organism')) AND cdss.person_id=p.person_id AND p.gender='M' 
                AND cdss.age_group_code=7)) AND concept_id = (select c.concept_id from concept c where c.description = '{concept_name}')""")

        sql_query = sql_query.format(**values_map)

        patient_cohort_concept = pd.read_sql(sql_query, con = conn)
        patient_cohort_concept.columns = [*patient_cohort_concept.columns[:-1], i]
    
        name = "patient_cohort_concept" + i
    
 
        locals()[name] = patient_cohort_concept.copy()

        dfs.append(locals()[name])
    
    
   
#     final_patient_cohort_concept.merge(patient_cohort_concept, on = 'person_id', how = 'outer')

#    final_patient_cohort_concept = pd.merge(patient_cohort_concept, final_patient_cohort_concept, on='person_id', how='inner')

#  l = [df.set_index('person_id') for df in dfs] 

    l = [df for df in dfs] 

# pd.DataFrame.join accepts a list of dataframes as 'other' 
    final_patient_cohort_concept = l[0].append(l[1:])


# final_patient_cohort_concept = l[0].append(l[1:])

# final_patient_cohort_concept = final_patient_cohort_concept.reset_index(drop = False)

    for column in final_patient_cohort_concept.columns:
        final_patient_cohort_concept[column] = final_patient_cohort_concept[column].fillna(final_patient_cohort_concept[column].mean())

        out = widgets.Output()
        with out:
            display(final_patient_cohort_concept)
        return out

# # Getting total count of the patient cohort as text widget

    total_count_cohort = len(final_patient_cohort_concept)
    text_patient_cohort_count = widgets.IntText(value = total_count_cohort , description= 'Total count:')
    display(text_patient_cohort_count)
    patient_cohort_concept_() 

In [1174]:
# getting the dropdown for concept_name

<div class="card" style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);max-width: 100%; margin: auto;padding:10px; font-family: arial;background:#D9D9D9">
    <p>Please choose one or multiple <b style="color:#195734">CONCEPT NAMES</b> to run your analysis on.For choosing multiple options,press CTRL:</p>
 </div>

In [1175]:
output_concept_name = widgets.Output()
# dropdown_concept_name = widgets.Dropdown(options = patient_attribute_df['description'].unique(), description = '<b>Concept Name:</b>')

dropdown_concept_name = widgets.SelectMultiple(options= patient_attribute_df['name'].unique(), style =style,disabled=False, rows = 10, size = 10)


def dropdown_concept_name_eventhandler(change):
    output_concept_name.clear_output()
    with output_concept_name:
        display(patient_attribute_df[patient_attribute_df["name"] == change.new])
    patient_cohort_concept_()
    on_3button_clicked(dropdown_concept_name.value)
    
def dropdown_concept_name_eventhandler1(change):
    output_concept_name.clear_output()
    with output_concept_name:
        display(patient_attribute_df[patient_attribute_df["name"] == change.new])

    
dropdown_concept_name.observe(dropdown_concept_name_eventhandler1, names='value')


# Button = widgets.Button(description='Choose Diagnosis', button_style='success', 
#                                    layout=Layout(width='200px'), tooltip='Choose Diagnosis', icon='check')
# display(Button.on_click(x))

display(dropdown_concept_name) 
# widgets.Button(description = 'Click after choosing Concept Name')


output = widgets.Output()
button =widgets.Button(description="Click after multiple selection!")
display(button, output)

button.on_click(dropdown_concept_name_eventhandler)


SelectMultiple(options=('mcv', 'monos', 'sodium, urine', 'white blood cells', 'cholesterol, pleural', 'chlorid…

Button(description='Click after multiple selection!', style=ButtonStyle())

Output()

/tmp/ipykernel_45791/1020584946.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patient_cohort_concept = pd.read_sql(sql_query, con = conn)


DatabaseError: Execution failed on sql: select person_id, value_numeric from obs where person_id IN (select p.person_id from person p, 
                mimic_openmrs_cdss cdss where p.person_id IN (select patient_id from encounter_diagnosis where 
                diagnosis_coded IN (select c.concept_id from concept c where c.description LIKE 
                ('Pneumonia, unspecified organism')) AND cdss.person_id=p.person_id AND p.gender='M' 
                AND cdss.age_group_code=7)) AND concept_id = (select c.concept_id from concept c where c.description = 'mcv')
(2013, 'Lost connection to MySQL server during query')
unable to rollback

/tmp/ipykernel_45791/1020584946.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patient_cohort_concept = pd.read_sql(sql_query, con = conn)


DatabaseError: Execution failed on sql: select person_id, value_numeric from obs where person_id IN (select p.person_id from person p, 
                mimic_openmrs_cdss cdss where p.person_id IN (select patient_id from encounter_diagnosis where 
                diagnosis_coded IN (select c.concept_id from concept c where c.description LIKE 
                ('Pneumonia, unspecified organism')) AND cdss.person_id=p.person_id AND p.gender='M' 
                AND cdss.age_group_code=7)) AND concept_id = (select c.concept_id from concept c where c.description = 'mcv')
(2013, 'Lost connection to MySQL server during query')
unable to rollback

In [1176]:
# getting values of the concept name of the cohort

In [1177]:
def conc():
    output_count = widgets.Output()
    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)
    concept_name

# concept_name = ['creatinine', 'albumin']
    values_map = {}
    a = 0
# final_patient_cohort_concept = pd.DataFrame(columns = ['person_id'])
# final_patient_cohort_concept['person_id'] = final_patient_cohort_concept['person_id'].astype(int)

    dfs = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1
    
        sql_query = ("""select person_id, value_numeric from obs where person_id IN (select p.person_id from person p, 
                mimic_openmrs_cdss cdss where p.person_id IN (select patient_id from encounter_diagnosis where 
                diagnosis_coded IN (select c.concept_id from concept c where c.description LIKE 
                ('Pneumonia, unspecified organism')) AND cdss.person_id=p.person_id AND p.gender='M' 
                AND cdss.age_group_code=7)) AND concept_id = (select c.concept_id from concept c where c.description = '{concept_name}')""")

        sql_query = sql_query.format(**values_map)

        patient_cohort_concept = pd.read_sql(sql_query, con = conn)
        patient_cohort_concept.columns = [*patient_cohort_concept.columns[:-1], i]
    
        name = "patient_cohort_concept" + i
    
        locals()[name] = patient_cohort_concept.copy()

        dfs.append(locals()[name])
    
    
   
#     final_patient_cohort_concept.merge(patient_cohort_concept, on = 'person_id', how = 'outer')

#    final_patient_cohort_concept = pd.merge(patient_cohort_concept, final_patient_cohort_concept, on='person_id', how='inner')

#  l = [df.set_index('person_id') for df in dfs] 

    l = [df for df in dfs] 

# pd.DataFrame.join accepts a list of dataframes as 'other' 
    final_patient_cohort_concept = l[0].append(l[1:])


# final_patient_cohort_concept = l[0].append(l[1:])

# final_patient_cohort_concept = final_patient_cohort_concept.reset_index(drop = False)

    for column in final_patient_cohort_concept.columns:
        final_patient_cohort_concept[column] = final_patient_cohort_concept[column].fillna(final_patient_cohort_concept[column].mean())

    def patient_cohort_concept_():
        out = widgets.Output()
        with out:
            display(final_patient_cohort_concept)
        return out
    return final_patient_cohort_concept

In [1178]:
## write a function to append each concepts and its values

In [1179]:
# get individual patient value

In [1180]:
def con():
    
    output_count = widgets.Output()

    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)

    # concept_name = ['creatinine', 'albumin']
    values_map = {}

    a = 0
    final_patient_ind_value_numeric = pd.DataFrame(columns = ['person_id'])
    final_patient_ind_value_numeric['person_id'] = final_patient_ind_value_numeric['person_id'].astype(int)

    dfss = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1

        patient_value_numeric = (""" select o.person_id, o.value_numeric from obs as o where o.concept_id IN (SELECT concept_id FROM concept WHERE description = '{concept_name}') AND o.person_id = {patient_id}""")
        patient_value_numeric = patient_value_numeric.format(**values_map)

        patient_ind_value_numeric = pd.read_sql(patient_value_numeric, con = conn)

        patient_ind_value_numeric.columns = [*patient_ind_value_numeric.columns[:-1], i]

        name = "patient_ind_value_numeric" + i

        locals()[name] = patient_ind_value_numeric.copy()

        dfss.append(locals()[name])

    ls = [df.set_index('person_id') for df in dfss]

    # pd.DataFrame.join accepts a list of dataframes as 'other'
    final_patient_ind_value_numeric = ls[0].append(ls[1:])
    # final_patient_ind_value_numeric = ls[0].join(ls[1:], how = 'inner')
    final_patient_ind_value_numeric = final_patient_ind_value_numeric.reset_index(drop = False)

    for column in final_patient_ind_value_numeric.columns:
        final_patient_ind_value_numeric[column] = final_patient_ind_value_numeric[column].fillna(final_patient_ind_value_numeric[column].mean())



#     def individual_patient_value():
#         display(final_patient_ind_value_numeric)


#     individual_patient_value() 

    return final_patient_ind_value_numeric

In [1181]:
#final_patient_ind_value_numeric

In [1182]:
from IPython.core.display import HTML as Center

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    border-radius: 16px;
}
</style> """)

In [1183]:
# the plot for distribution

In [1184]:
def showplot():
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value
    plot_output = widgets.Output()
    plot_output.clear_output()
    concept_name = list(concept_name)
    with plot_output:
        for column in concept_name:
#         with plot_output:
            fig, ax = plt.subplots(figsize=(3, 2))
            sns.kdeplot(data=final_patient_cohort_concept[column],
                color='blue', label='cohort', fill=True, ax=ax)
            sns.kdeplot(data=final_patient_ind_value_numeric[column],
                color='yellow', label='patient', fill=True, ax=ax)
            ax.legend()
            plt.title("Normality Distribution")
            plt.tight_layout()
            plt.show()

    display(plot_output)
    #return plot_output
    return
    showplot()

In [1185]:
# Statistical test
# Shapiro-Wilk Test Statistic

In [1186]:
def shapiro_test():
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value
    shapiro_output = widgets.Output()
    shapiro_output.clear_output()
    #with shapiro_output:
    cohort = final_patient_cohort_concept
    patient = final_patient_ind_value_numeric
    rowLabels = concept_name #substitue with concept name
    colLabels = ['Shapiro-Wilk Test Statistic', 'p-value']
    rowLabels_ttest = concept_name
    colLabels_ttest = ['T-statistic', 'p-value']

    plt.figure()

    cell_text = []
    results_ttest =[]
    for j in concept_name:
        shapiro = stats.shapiro(cohort[j])
        cell_text.append(shapiro)
        if shapiro[1] < 0.05:
            mann_u = stats.mannwhitneyu(cohort[j], patient[j], alternative='two-sided')
            results_ttest.append(mann_u)
        else:
            t_test = stats.ttest_ind((cohort.loc[:,j]), (patient.loc[:,j]), equal_var = False)
            results_ttest.append(t_test)
        #print(cohort.loc[:,j])
            

    fig, ax = plt.subplots(2,1, figsize=(14, 22))
    #     figure, axis = plt.subplots(2, 2)
    a = ax[0].table(rowLabels=rowLabels, colLabels=colLabels, cellText=cell_text, cellLoc= 'center', loc='center')
    a.set_fontsize(12)
    a.scale(1, 2)
    ax[0].axis('off')

    b = ax[1].table(rowLabels=rowLabels_ttest, colLabels=colLabels_ttest, cellText=results_ttest, cellLoc= 'center', loc='center')
    b.set_fontsize(12)
    b.scale(1, 2)
    ax[1].axis('off')

    ax[0].set_title('Shapiro Wilk Test')
    ax[1].set_title('Statistical Test')
    
    with shapiro_output:
        plt.show()
        
    return shapiro_output      
    shapiro_test()

In [1187]:
## checking mean of diagnosis for each patient

In [1188]:
encounter_diagnosis = pd.read_sql('select patient_id, diagnosis_coded from encounter_diagnosis', con=conn)
encounter_diagnosis_count = encounter_diagnosis.groupby("patient_id")["diagnosis_coded"].count()

/tmp/ipykernel_45791/1660638309.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  encounter_diagnosis = pd.read_sql('select patient_id, diagnosis_coded from encounter_diagnosis', con=conn)


In [1189]:
top_20_diag = pd.DataFrame(encounter_diagnosis['diagnosis_coded'].value_counts().sort_index(ascending=False).sort_values(ascending=False).head(25))
top_20_diag = top_20_diag.reset_index()
list_top_20_diag = list(top_20_diag['index'])
#list_top_20_diag

In [1190]:
## Obtaining class labels ##

In [1191]:
class_labels = pd.read_sql("SELECT description from concept where concept_id IN (SELECT * FROM(SELECT diagnosis_coded FROM encounter_diagnosis GROUP BY diagnosis_coded ORDER BY COUNT(*) DESC LIMIT 25) as xyz)", con=conn) 
#class_labels                      

/tmp/ipykernel_45791/1148437002.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  class_labels = pd.read_sql("SELECT description from concept where concept_id IN (SELECT * FROM(SELECT diagnosis_coded FROM encounter_diagnosis GROUP BY diagnosis_coded ORDER BY COUNT(*) DESC LIMIT 25) as xyz)", con=conn)


In [1192]:
old_diagnosis_name = None
#run this cell only when we restart the kernel


<div class="card" style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);max-width: 100%; margin: auto;padding:10px; font-family: arial; background:#D9D9D9">
    <p>The Following Dropdown asks you for the input of risks from the available risks you want to give for running analysis on any <b style="color:#195734"> ONE DIAGNOSIS</b>: </p>
 </div>

In [1193]:
output_diag_name = widgets.Output()

dropdown_diagnosis_name = widgets.Dropdown(options= class_labels['description'].unique(), style = style, description='Risk of diagnosis for the patient:',disabled=False, rows = 15, size = 10, layout=Layout(width = '450px'))

def dropdown_diagnosis_name_eventhandler(change):
    output_diag_name.clear_output()
    with output_diag_name:
        display(class_labels[class_labels["description"] == change.new])
        diagnosis_name = dropdown_diagnosis_name.value
    print(diagnosis_name)    
    dashboard(dropdown_diagnosis_name.value)
    
    
     
dropdown_diagnosis_name.observe(dropdown_diagnosis_name_eventhandler, names='value')
    
display(dropdown_diagnosis_name) 


Dropdown(description='Risk of diagnosis for the patient:', layout=Layout(width='450px'), options=('Single live…

In [1194]:
diagnosis_name = dropdown_diagnosis_name.value
values_map3 = {}
values_map3['diagnosis_name'] = diagnosis_name
values_map3
get_patient_id = ("""select patient_id from encounter_diagnosis where diagnosis_coded IN (select * from (select concept_id from concept where description = "{diagnosis_name}") as pqr )""") 
get_patient_id = get_patient_id.format(**values_map3) 
get_patient_id = pd.read_sql(get_patient_id, con=conn)  
#get_patient_id

/tmp/ipykernel_45791/1255386782.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  get_patient_id = pd.read_sql(get_patient_id, con=conn)


In [1195]:
#diagnosis_name

In [1196]:
## Adding class labels to cohort dataframe

#final_patient_cohort_concept[diagnosis_name] = ''
def final_patient_cohort_concept_1(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    values_map3 = {}
    values_map3['diagnosis_name'] = diagnosis_name
    values_map3
    get_patient_id = ("""select patient_id from encounter_diagnosis where diagnosis_coded IN (select * from (select concept_id from concept where description = "{diagnosis_name}") as pqr )""") 
    get_patient_id = get_patient_id.format(**values_map3) 
    get_patient_id = pd.read_sql(get_patient_id, con=conn)  
    #get_patient_id
    
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_cohort_concept=conc()

    for i, row in final_patient_cohort_concept.iterrows():
        if row['person_id'] in get_patient_id['patient_id'].tolist():

            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 1
        else:
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 0


    return final_patient_cohort_concept    

In [1197]:
diagnosis_name = dropdown_diagnosis_name.value
if old_diagnosis_name != diagnosis_name:
    try:
        final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
        final_patient_cohort_concept.drop([old_diagnosis_name], axis =1, inplace=True)
    except Exception as ex:
        #print(ex)
        pass
    old_diagnosis_name = diagnosis_name

/tmp/ipykernel_45791/1197646972.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  get_patient_id = pd.read_sql(get_patient_id, con=conn)


In [1198]:
#final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
# final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
# #final_patient_cohort_concept_feat

In [1199]:
## correlation 

In [1200]:
diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1201]:
def on_corrButton_clicked(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value
    for i, row in final_patient_cohort_concept.iterrows():
        if row['person_id'] in get_patient_id['patient_id'].tolist():
      
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 1
        else:
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 0
    
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    corr_output=widgets.Output()
    
    with corr_output:

        plt.figure(figsize = (20,15))
        corr = final_patient_cohort_concept_feat.corr(method='spearman')


        ax = sns.heatmap(corr, annot=True, fmt ='.1g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')
            
        
        plt.show(ax)
    return corr_output
    
corrButton = widgets.Button(description='Show Correlation', button_style='danger',layout=Layout(width='200px'), tooltip='Show Correlation', icon='check')


In [1202]:
diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1203]:
def on_assocButton_clicked(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value
    for i, row in final_patient_cohort_concept.iterrows():
        if row['person_id'] in get_patient_id['patient_id'].tolist():
      
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 1
        else:
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 0
    concept_name = dropdown_concept_name.value
    
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    feature_columns = list(final_patient_cohort_concept.columns[:-1])
    target_columns = final_patient_cohort_concept[diagnosis_name]
    assoc_output = widgets.Output()
    assoc_output.clear_output()
    with assoc_output:
        colLabels = ['Statistic','p-value']
        rowLabels = feature_columns
        plt.figure() 
        cell_Text = []
        for j in feature_columns:
            kruskaltest = stats.kruskal(target_columns, final_patient_cohort_concept[j])
            cell_Text.append(kruskaltest)
        table = plt.table(rowLabels=rowLabels, colLabels=colLabels, cellText=cell_Text, cellLoc= 'center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        #table.scale(2, 2)
#         table.set_title('Kruskal-wallis Test')
        plt.axis('off')
        for (row, col), cell in table.get_celld().items():
            try:
                if col==1 and row>=1 and float(cell.get_text()) < 0.05:
                    cell.set_text_props(fontproperties=FontProperties(weight='bold'))
            except:
                pass
        
        plt.show()
    #return assoc_output,final_patient_cohort_concept_feat
    return assoc_output

    for i in range(len(out)):
            tab.set_title(i, impression[i])

            tab.children = out
            display(tab)
            display(regressionButton)

assocButton = widgets.Button(description='Show Association (Kruskal-wallis)', button_style='danger', 
                               layout=Layout(width='350px'), tooltip='Show Association (Kruskal-wallis)', icon='check')
assocButton.on_click(on_assocButton_clicked)

In [1204]:
diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1205]:
def on_regressionButton_clicked(diagnosis_name):

    diagnosis_name = dropdown_diagnosis_name.value
    display(Markdown("## RandomForest Classifier Result for:"))
    print(diagnosis_name)
#     values_map3 = {}
#     values_map3['diagnosis_name'] = diagnosis_name
#     values_map3
#     get_patient_id = ("""select patient_id from encounter_diagnosis where diagnosis_coded IN (select * from (select concept_id from concept where description = "{diagnosis_name}") as pqr )""") 
#     get_patient_id = get_patient_id.format(**values_map3) 
#     get_patient_id = pd.read_sql(get_patient_id, con=conn)  
    #get_patient_id
    
    
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=final_patient_cohort_concept_1(dropdown_diagnosis_name.value)
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    
    diagnosis_name = dropdown_diagnosis_name.value
#     old_diagnosis_name = None
#     if old_diagnosis_name != diagnosis_name:
#         try:
#             final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
#             final_patient_cohort_concept.drop([old_diagnosis_name], axis =1, inplace=True)
#         except Exception as ex:
#             #print(ex)
#             pass
#         old_diagnosis_name = diagnosis_name

    
    #final_patient_cohort_concept_feat
    dict_pred = {1: 'Diagnosis Present', 0: 'Diagnosis Not Present'}
    import time
    start_time = time.time()

    patient_feat = (final_patient_ind_value_numeric.iloc[:1]).drop(columns = ['person_id']).values

    predictor = final_patient_cohort_concept_feat.iloc[:, :-1].values
    target = (final_patient_cohort_concept_feat[[diagnosis_name]]).values
    #display(final_patient_cohort_concept_feat)
    #display(final_patient_cohort_concept[["Acute kidney failure, unspecified"]].value_counts())
    X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.25, random_state=42,stratify=target)
    #print(y_train)




    # transform the dataset
    oversample = SMOTE(random_state=100)
    X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)


    # Feature Scaling

    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train_res)
    X_test = sc_X.transform(X_test)
    
    # Split the data into train and test
    # def on_regressionButton_clicked():

    # Fitting Random Forest classifier to the Training set
    Rfclassifier = RandomForestClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=0)

    Rfclassifier.fit(X_train_res, y_train_res.ravel())

    # Predicting the Test set results
    y_pred_rf = Rfclassifier.predict(X_test)
    #print('y_preds_rf:', y_pred_rf)

    # # accuracy
    # accuarcy_rf = accuracy_score(y_test, y_pred_rf)
    # print('accuarcy_rf:', accuarcy_rf)

    # Making the confusion Matrix
    cm_rf = confusion_matrix(y_test, y_pred_rf)
    print('confusion matrix_rf:', cm_rf)

    # predict for the patient
    patient_pred_rf = Rfclassifier.predict(patient_feat)
    text_prediction_rf = widgets.Text(value = dict_pred[int(patient_pred_rf[0])], description= 'Prediction:', style =style)
    def rf_report():
        rf_report_output = widgets.Output() 
        rf_report_output.clear_output()
        #with rf_report_output:
        patient_pred_rf = Rfclassifier.predict(patient_feat)
        print('patient_pred_rf:', dict_pred[int(patient_pred_rf[0])])
        text_prediction_rf = widgets.Text(value = dict_pred[int(patient_pred_rf[0])], description= 'Prediction:', style =style)
        report = classification_report(y_test, y_pred_rf, output_dict=True)
        df_report = pd.DataFrame(report).transpose()


        with rf_report_output:
            display(df_report)

        items = [text_prediction_rf, rf_report_output]
        box = Box(children=items)
        return box


    display(rf_report())
    return

    for i in range(len(out)):
        tab.set_title(i, impression[i])

    tab.children = out
    display(tab)
    display(clusteringButton)

regressionButton = widgets.Button(description='Perform Linear Regression', button_style='danger', 
                               layout=Layout(width='300px'), tooltip='Perform Linear Regression', icon='check')
regressionButton.on_click(on_regressionButton_clicked)
    #print("--- %s seconds ---" % (time.time() - start_time))

In [1206]:
diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1207]:
def on_lightGBMButton_clicked(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    display(Markdown("## LightGBM Classifier Result is as follows:"))
    print(diagnosis_name)
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=final_patient_cohort_concept_1(dropdown_diagnosis_name.value)
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    dict_pred = {1: 'Diagnosis Present', 0: 'Diagnosis Not Present'}
    import time
    start_time = time.time()
    final_patient_ind_value_numeric=con()
    patient_feat = (final_patient_ind_value_numeric.iloc[:1]).drop(columns = ['person_id']).values

    predictor = final_patient_cohort_concept_feat.iloc[:, :-1].values
    target = (final_patient_cohort_concept_feat[[diagnosis_name]]).values

    X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.25, random_state=42)

    # transform the dataset
    oversample = SMOTE()
    X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

    # Feature Scaling
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train_res)
    X_test = sc_X.transform(X_test)

    # Split the data into train and test
    # def on_regressionButton_clicked():

    # Split the data into train and test
    # def on_regressionButton_clicked():


    # Fitting Random Forest classifier to the Training set
    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(X_train_res, y_train_res)

    # Predicting the Test set results
    y_pred_lgb = lgb_classifier.predict(X_test)
    print('y_preds_lgb:', y_pred_lgb)

    # accuracy
    # accuarcy_lgb = accuracy_score(y_test, y_pred_lgb)
    # print('accuarcy_rf:', accuarcy_lgb)

    # Making the confusion Matrix
    cm_lgb = confusion_matrix(y_test, y_pred_lgb)
    print('confusion matrix_lgb:', cm_lgb)

    patient_pred_lgb = lgb_classifier.predict(patient_feat)
    text_prediction_lgb = widgets.Text(value = dict_pred[int(patient_pred_lgb[0])], description= 'Prediction:', style =style)

    def lightGBM_report():
        lightGBM_report_output = widgets.Output() 
        lightGBM_report_output.clear_output()

        patient_pred_lgb = lgb_classifier.predict(patient_feat)
        print('patient_pred_lgb:', dict_pred[int(patient_pred_lgb[0])])
        text_prediction_lgb = widgets.Text(value = dict_pred[int(patient_pred_lgb[0])], description= 'Prediction:', style =style)
        report = classification_report(y_test, y_pred_lgb, output_dict=True)
        df_report = pd.DataFrame(report).transpose()

        with lightGBM_report_output:
            display(df_report)

        items = [text_prediction_lgb, lightGBM_report_output]
        box = Box(children=items)
        return box


    display(lightGBM_report())

    return
    on_lightGBMButton_clicked(dropdown_diagnosis_name.value)

In [1208]:
#on_catboostButton_clicked()

In [1209]:
diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1210]:
# ## CatBoost model fitting

# # Applying grid search to find the best model and best parameters

# import time
# # Split the data into train and test
# def on_catboostButton_clicked(diagnosis_name):
   
#     final_patient_cohort_concept=conc()
#     diagnosis_name = dropdown_diagnosis_name.value
#     dict_pred = {1: 'Diagnosis Present', 0: 'Diagnosis Not Present'}
#     final_patient_ind_value_numeric=con()
#     final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    
    
#     # Split the data into train and test
#     # def on_regressionButton_clicked():

#     # Fitting Random Forest classifier to the Training set
#     Rfclassifier = RandomForestClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=0)

#     Rfclassifier.fit(X_train_res, y_train_res.ravel())


#     # predict for the patient
 
#     start_time = time.time()

#     patient_feat = (final_patient_ind_value_numeric.iloc[:1]).drop(columns = ['person_id']).values

#     predictor = final_patient_cohort_concept_feat.iloc[:, :-1].values
#     target = (final_patient_cohort_concept_feat[[diagnosis_name]]).values
#     X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.25, random_state=42)

#  # Feature Scaling

#     sc_X = StandardScaler()
#     X_train = sc_X.fit_transform(X_train)
#     X_test = sc_X.transform(X_test)

# # # Fitting Random Forest classifier to the Training set
#     rfc = CatBoostClassifier(iterations=10)
#     error_score='raise'

#     forest_params = [{'max_depth': list(range(10, 15)), 'max_features': list(range(0,14))}]

#     clf = GridSearchCV(rfc, forest_params, cv = 5, scoring='accuracy')

#     model = clf.fit(X_train, y_train.ravel())

#     print(model.best_params_)

#     print(model.best_score_)


#     print(model.best_estimator_)

#     pred = model.best_estimator_.predict(X_test)
#     print('prediction:', pred)

#     patient_pred = model.best_estimator_.predict(patient_feat)
#     print('patient_pred_rf:', patient_pred_rf[0])


#     print(classification_report(y_test, pred))
    
#     return

# on_catboostButton_clicked(diagnosis_name)

In [1211]:
# diagnosis_name = dropdown_diagnosis_name.value
#diagnosis_name

In [1212]:
def auc(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    display(Markdown("## AUC Results are as follows:"))
    print(diagnosis_name)
#     values_map3 = {}
#     values_map3['diagnosis_name'] = diagnosis_name
#     values_map3
#     get_patient_id = ("""select patient_id from encounter_diagnosis where diagnosis_coded IN (select * from (select concept_id from concept where description = "{diagnosis_name}") as pqr )""") 
#     get_patient_id = get_patient_id.format(**values_map3) 
#     get_patient_id = pd.read_sql(get_patient_id, con=conn)  
    #get_patient_id
     
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=final_patient_cohort_concept_1(dropdown_diagnosis_name.value)
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    
    diagnosis_name = dropdown_diagnosis_name.value
#     old_diagnosis_name = None
#     if old_diagnosis_name != diagnosis_name:
#         try:
#             final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
#             final_patient_cohort_concept.drop([old_diagnosis_name], axis =1, inplace=True)
#         except Exception as ex:
#             #print(ex)
#             pass
#         old_diagnosis_name = diagnosis_name   
    #final_patient_cohort_concept_feat
    dict_pred = {1: 'Diagnosis Present', 0: 'Diagnosis Not Present'}
    import time
    start_time = time.time()

    patient_feat = (final_patient_ind_value_numeric.iloc[:1]).drop(columns = ['person_id']).values

    predictor = final_patient_cohort_concept_feat.iloc[:, :-1].values
    target = (final_patient_cohort_concept_feat[[diagnosis_name]]).values
    #display(final_patient_cohort_concept_feat)
    #display(final_patient_cohort_concept[["Acute kidney failure, unspecified"]].value_counts())
    X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.25, random_state=42,stratify=target)
    #print(y_train)

    # transform the dataset
    oversample = SMOTE(random_state=100)
    X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)


    # Feature Scaling
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train_res)
    X_test = sc_X.transform(X_test)
     
    Rfclassifier = RandomForestClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=0)
    Rfclassifier.fit(X_train_res, y_train_res.ravel())

    # Split the data into train and test
    # def on_regressionButton_clicked():

    # Split the data into train and test
    # def on_regressionButton_clicked():


    # Fitting Random Forest classifier to the Training set

    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(X_train_res, y_train_res)

    # Predicting the Test set results
    y_pred_lgb = lgb_classifier.predict(X_test)

    # accuracy
    # accuarcy_lgb = accuracy_score(y_test, y_pred_lgb)
    # print('accuarcy_rf:', accuarcy_lgb)

    # Making the confusion Matrix
    cm_lgb = confusion_matrix(y_test, y_pred_lgb)

    patient_pred_lgb = lgb_classifier.predict(patient_feat)
    text_prediction_lgb = widgets.Text(value = dict_pred[int(patient_pred_lgb[0])], description= '<b>Prediction:</b>', style =style)

    auc_rf = widgets.Output() 
    auc_rf.clear_output()
    
#     auc_gbm = widgets.Output()
#     auc_gbm.clear_output()
    
    predictor_prob_rf = Rfclassifier.predict_proba(X_test)
    x_rf = skplt.metrics.plot_precision_recall_curve(y_test, predictor_prob_rf)
    
    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(X_train_res, y_train_res)
    
    predictor_prob_gbm = lgb_classifier.predict_proba(X_test)
    x_gbm = skplt.metrics.plot_precision_recall_curve(y_test, predictor_prob_gbm)
    
    
    with auc_rf:
        plt.show(x_rf)
        plt.show(x_gbm)
        skplt.metrics.plot_precision_recall_curve(y_test, predictor_prob_rf).plot()
        x_gbm.plot()
    items = [auc_rf]
    box = Box(children=items)
    display(auc_rf)
    
    return 
    auc(diagnosis_name)    

In [1213]:
# # Fastai
# from fastai.vision import *

In [1214]:
def fastai_report(diagnosis_name): 
    
    import time
    diagnosis_name = dropdown_diagnosis_name.value
    display(Markdown("## FASTAI Report for:"))
    print(diagnosis_name)
#values_map3 = {}
#     values_map3['diagnosis_name'] = diagnosis_name
#     values_map3
#     get_patient_id = ("""select patient_id from encounter_diagnosis where diagnosis_coded IN (select * from (select concept_id from concept where description = "{diagnosis_name}") as pqr )""") 
#     get_patient_id = get_patient_id.format(**values_map3) 
#     get_patient_id = pd.read_sql(get_patient_id, con=conn)  
    #get_patient_id
     
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=final_patient_cohort_concept_1(dropdown_diagnosis_name.value)
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    
    start_time = time.time()

    cont_names = list(final_patient_cohort_concept_feat.columns[:-1])

    norm = Normalize()
    to = TabularPandas(final_patient_cohort_concept_feat, norm, cont_names=cont_names)
    norms = to.procs.normalize

    procs = [Categorify, FillMissing, Normalize]
    cont_names = list(final_patient_cohort_concept_feat.columns[:-1])

    y_names = final_patient_cohort_concept_feat.columns[-1]
    y_block = CategoryBlock()

    splits = RandomSplitter()(range_of(final_patient_cohort_concept_feat))

    to = TabularPandas(final_patient_cohort_concept_feat, procs=[], cont_names=cont_names,
                       y_names=y_names, y_block=y_block, splits=splits)

    dls = to.dataloaders()
    trn_dl = TabDataLoader(to.train, bs=64, shuffle=True, drop_last=True)
    val_dl = TabDataLoader(to.valid, bs=128)
    dls = DataLoaders(trn_dl, val_dl)

    emb_szs = get_emb_sz(to); emb_szs
    cont_len = len(to.cont_names); cont_len
    batch = dls.one_batch(); len(batch)
    batch[0][0], batch[1][0]
    net = TabularModel(emb_szs, cont_len, 2, [200,100])

    learn = tabular_learner(dls, [200,100], metrics=accuracy)
    learn.lr_find() 

    dls = to.dataloaders(bs=1024)
    learn = tabular_learner(dls, [200,100], metrics=accuracy)
    learn.fit_one_cycle(10,cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=3))

    def fastai_report(diagnosis_name):
        fastai_report_output = widgets.Output() 
        fastai_report_output.clear_output()

    #patient_pred_lgb = lgb_classifier.predict(patient_feat)
    #print('patient_pred_lgb:', dict_pred[int(patient_pred_lgb[0])])
    #text_prediction_fastai = widgets.Text(value = dict_pred[int(patient_pred_lgb[0])], description= '<b>Prediction:</b>', style =style)
        interpret = ClassificationInterpretation.from_learner(learn)
        interpret = interpret.print_classification_report()
        df_report = pd.DataFrame(interpret).transpose()

        with fastai_report_output:
            display(df_report)

        items = [fastai_report_output]
        box = Box(children=items)
        return box

    
    fastai_report(diagnosis_name)

# interpret = ClassificationInterpretation.from_learner(learn)
# interpret.print_classification_report()

# print("--- %s seconds ---" % (time.time() - start_time))

# interpret = ClassificationInterpretation.from_learner(learn)
# interpret.print_classification_report()

# print("--- %s seconds ---" % (time.time() - start_time))

In [1215]:
# Building the dashboard

In [1216]:
# def dashboard():
#     item_layout = widgets.Layout(margin='0 0 50px 0')
    
#     input_widgets = widgets.HBox([dropdown_diagnames_pat_id, dropdown_concept_name, dropdown_diagnosis_name], layout = item_layout )

    
#     tab = widgets.Tab([patient_cohort_concept_(), showplot(), shapiro_test(), on_corrButton_clicked(), on_assocButton_clicked(), on_Regressionbutton(), lightGBM_report(), auc()],layout=item_layout)
#     tab.set_title(0, 'Dataset Exploration')
#     tab.set_title(1, 'Distribution Plot')
#     tab.set_title(2, 'Statistical Test')
#     tab.set_title(3, 'Correlation Test')
#     tab.set_title(4, 'Association Test')
#     tab.set_title(5, 'Random Forest Classifier')
#     tab.set_title(6, 'LightGBM Classifier')
#     tab.set_title(7, 'AUC_ROC Plot')
    
#     dashboard = widgets.VBox([input_widgets, tab])
# display(dashboard)

In [1217]:
def dashboard(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    item_layout = widgets.Layout(margin='0 0 50px 0')
    
    input_widgets = widgets.HBox([dropdown_diagnames_pat_id, dropdown_concept_name, dropdown_diagnosis_name], layout = item_layout )

    
    #tab = widgets.Tab([patient_cohort_concept_(), showplot(), shapiro_test(), on_corrButton_clicked(diagnosis_name), on_assocButton_clicked(diagnosis_name), on_regressionButton_clicked(diagnosis_name),on_lightGBMButton_clicked(dropdown_diagnosis_name.value)],layout=item_layout)
    tab = widgets.Tab([patient_cohort_concept_(), showplot(), shapiro_test(), on_corrButton_clicked(diagnosis_name), on_assocButton_clicked(diagnosis_name), on_regressionButton_clicked(diagnosis_name),on_lightGBMButton_clicked(diagnosis_name),auc(diagnosis_name),fastai_report(diagnosis_name)],layout=item_layout)
    tab.set_title(0, 'Dataset Exploration')
    tab.set_title(1, 'Distribution Plot')
    tab.set_title(2, 'Statistical Test')
    tab.set_title(3, 'Correlation Test')
    tab.set_title(4, 'Association Test')
    tab.set_title(5, 'Random Forest Classifier')
    tab.set_title(6, 'LightGBM Classifier')
    #tab.set_title(7, 'FastAI Classifier')
    
    dashboard = widgets.VBox([input_widgets, tab])
    display(dashboard)
    